# Problem 1 (MATLAB neural network toolbox) 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import math
import numpy as np

### Data and Model Prediction

In [2]:
X = torch.unsqueeze(torch.arange(0, 2, 0.01), dim = 1)
y = torch.sin(2 * math.pi * X) + 0.1 * torch.rand(X.size())


In [3]:
class SimpleMLP(nn.Module):
    def __init__(self):
        super(SimpleMLP, self).__init__()
        self.hidden = nn.Linear(1, 10) # input layer
        self.output = nn.Linear(10, 1) # output layer
        self.tan = nn.Tanh()
    def forward(self, x):
        x = self.tan(self.hidden(x))
        x = self.output(x)
        return x

model = SimpleMLP()
criterion = nn.SmoothL1Loss()
optimizer = optim.Adam(model.parameters(), lr=0.044)

loss_threshold = 0.001
epochs = 10000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()
    
    if loss < loss_threshold:
        print("In my NN, root-mean-squared Error is below loss threshold at epoch", epoch + 1)
        break

predicted_train = model(X)
rms_error = torch.sqrt(torch.mean((predicted_train - y) ** 2)).item()
print("The root-mean-squared Error = ", rms_error)



In my NN, root-mean-squared Error is below loss threshold at epoch 919
The root-mean-squared Error =  0.0446290485560894


# Problem 2  (Convolutional neural network) 

In [19]:
import os
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision

ModuleNotFoundError: No module named 'torch.nn'

In [ ]:
EPOCH = 2                          # train the training data n times
BATCH_SIZE = 100
learning_rate = 0.001              # learning rate


# Mnist digits dataset  
if not(os.path.exists('./mnist/')) or not os.listdir('./mnist/'):
    # not mnist dir or mnist is empyt dir
    DOWNLOAD_MNIST = True
# training data
train_data = torchvision.datasets.MNIST(
    root='./mnist/',
    train=True,                                     
    transform=torchvision.transforms.ToTensor(),    # normalize in the range [0.0, 1.0]
                                                    
    download=True,
)


print(train_data.train_data.size())                 # ([60000, 28, 28])
print(train_data.train_labels.size())               # ([60000])
# image batch shape (6000, 1, 28, 28)
train_loader = Data.DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

# 2000 samples testing data
test_data = torchvision.datasets.MNIST(root='./mnist/', train=False)
test_x = torch.unsqueeze(test_data.test_data, dim=1).type(torch.FloatTensor)[:2000]/255.   # shape from (2000, 28, 28) to (2000, 1, 28, 28), value in range(0,1)
test_y = test_data.test_labels[:2000]

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (1, 28, 28)
            nn.Conv2d(
                in_channels=1,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # padding=(kernel_size-1)/2
            ),                              # output shape (16, 28, 28)
            nn.BatchNorm2d(16),             # Batch Normalization
            nn.ReLU(),                      # activation
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (16, 14, 14)
        )
        self.conv2 = nn.Sequential(         # input shape (16, 14, 14)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (32, 14, 14)
            nn.BatchNorm2d(32),             # Batch Normalization
            nn.ReLU(),                      # activation
            nn.MaxPool2d(2),                # output shape (32, 7, 7)
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(32 * 7 * 7, 256),     # fully connected layer, output 256 classes
            nn.ReLU(),                      # activation
        )
        self.out = nn.Linear(256, 10)       # fully connected layer, output 10 classes

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.flatten(1)                    # flatten the output to (batch_size, 32 * 7 * 7)
        x = self.fc_layers(x)
        output = self.out(x)
        output = nn.functional.softmax(output, dim=1)   # softmax
        return output
